In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets.mnist import MNIST
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch import optim
import time
from tqdm import tqdm
import os
from torch.utils.data import Dataset
from PIL import Image

In [19]:
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1" 

In [4]:
train_path = '/home/ielab/dataset/ms1m_dataset/train_mosaic/'
test_path = '/home/ielab/dataset/ms1m_dataset/test_masked'

In [5]:
col_list = os.listdir(train_path)

lb = {string : i for i,string in enumerate(col_list)}

In [6]:
col_list

['1977',
 '2782',
 '1236',
 '2546',
 '1867',
 '1647',
 '1458',
 '1686',
 '2647',
 '1504',
 '1148',
 '2373',
 '2246',
 '3262',
 '407',
 '1569',
 '226',
 '1663',
 '3590',
 '172',
 '1991',
 '192',
 '3183',
 '183',
 '1170',
 '1191',
 '256',
 '1618',
 '1731',
 '2076',
 '203',
 '3264',
 '1791',
 '2709',
 '2362',
 '2053',
 '2657',
 '3002',
 '1525',
 '2954',
 '2820',
 '2903',
 '3372',
 '2719',
 '1141',
 '2222',
 '2730',
 '265',
 '348',
 '1944',
 '3036',
 '1443',
 '1039',
 '3522',
 '1414',
 '1425',
 '1473',
 '1697',
 '2724',
 '1902',
 '1355',
 '2385',
 '466',
 '2276',
 '1015',
 '3338',
 '1481',
 '1433',
 '2144',
 '3578',
 '2529',
 '1486',
 '2157',
 '3501',
 '1259',
 '3095',
 '2240',
 '2883',
 '1915',
 '1004',
 '1107',
 '472',
 '2467',
 '1490',
 '2391',
 '148',
 '3345',
 '2387',
 '413',
 '2111',
 '2326',
 '2694',
 '2840',
 '1285',
 '2279',
 '1658',
 '3350',
 '3524',
 '1442',
 '1570',
 '2428',
 '2079',
 '2088',
 '1153',
 '2571',
 '3541',
 '1718',
 '2159',
 '2539',
 '2744',
 '2244',
 '1319',
 '336

In [7]:
lb

{'1977': 0,
 '2782': 1,
 '1236': 2,
 '2546': 3,
 '1867': 4,
 '1647': 5,
 '1458': 6,
 '1686': 7,
 '2647': 8,
 '1504': 9,
 '1148': 10,
 '2373': 11,
 '2246': 12,
 '3262': 13,
 '407': 14,
 '1569': 15,
 '226': 16,
 '1663': 17,
 '3590': 18,
 '172': 19,
 '1991': 20,
 '192': 21,
 '3183': 22,
 '183': 23,
 '1170': 24,
 '1191': 25,
 '256': 26,
 '1618': 27,
 '1731': 28,
 '2076': 29,
 '203': 30,
 '3264': 31,
 '1791': 32,
 '2709': 33,
 '2362': 34,
 '2053': 35,
 '2657': 36,
 '3002': 37,
 '1525': 38,
 '2954': 39,
 '2820': 40,
 '2903': 41,
 '3372': 42,
 '2719': 43,
 '1141': 44,
 '2222': 45,
 '2730': 46,
 '265': 47,
 '348': 48,
 '1944': 49,
 '3036': 50,
 '1443': 51,
 '1039': 52,
 '3522': 53,
 '1414': 54,
 '1425': 55,
 '1473': 56,
 '1697': 57,
 '2724': 58,
 '1902': 59,
 '1355': 60,
 '2385': 61,
 '466': 62,
 '2276': 63,
 '1015': 64,
 '3338': 65,
 '1481': 66,
 '1433': 67,
 '2144': 68,
 '3578': 69,
 '2529': 70,
 '1486': 71,
 '2157': 72,
 '3501': 73,
 '1259': 74,
 '3095': 75,
 '2240': 76,
 '2883': 77,
 '1915

In [8]:
batch_size = 64

In [9]:
num_classe = len(col_list)

In [10]:
trans = transforms.Compose([
    transforms.Resize((112,112)),
    transforms.ToTensor(),
    ])

In [11]:
class CustomDataset(Dataset):
    def __init__(self, img_dir, transform=None, target_transform=None):
        self.img_dir = img_dir
        self.dir_list = os.listdir(img_dir)
        self.transform = transform
        self.target_transform = target_transform

        self.files = []
        for folder in self.dir_list:
          folder_list = os.listdir(os.path.join(img_dir, folder))
          for file in folder_list:
            self.files.append([
                os.path.join(self.img_dir, folder+'/'+file),
                folder])

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        image = Image.open(self.files[idx][0])
        image = image.convert('RGB')
        image = self.transform(image)

        label = self.files[idx][1]
        label = lb[label]

        return image, label

In [12]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

trainset = CustomDataset(train_path, trans)
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4)

testset = CustomDataset(test_path, trans)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=4)

In [13]:
class ArcFaceLoss(nn.Module):
    def __init__(self, num_classes, embedding_size, margin, scale):
        """
        ArcFace: Additive Angular Margin Loss for Deep Face Recognition
        (https://arxiv.org/pdf/1801.07698.pdf)
        Args:
            num_classes: The number of classes in your training dataset
            embedding_size: The size of the embeddings that you pass into
            margin: m in the paper, the angular margin penalty in radians
            scale: s in the paper, feature scale
        """
        super().__init__()
        self.num_classes = num_classes
        self.embedding_size = embedding_size
        self.margin = margin
        self.scale = scale
        
        self.W = torch.nn.Parameter(torch.Tensor(num_classes, embedding_size))
        nn.init.xavier_normal_(self.W)
        
    def forward(self, embeddings, labels):
        """
        Args:
            embeddings: (None, embedding_size)
            labels: (None,)
        Returns:
            loss: scalar
        """
        cosine = self.get_cosine(embeddings) # (None, n_classes)
        mask = self.get_target_mask(labels) # (None, n_classes)
        cosine_of_target_classes = cosine[mask == 1] # (None, )
        modified_cosine_of_target_classes = self.modify_cosine_of_target_classes(
            cosine_of_target_classes
        ) # (None, )
        diff = (modified_cosine_of_target_classes - cosine_of_target_classes).unsqueeze(1) # (None,1)
        logits = cosine + (mask * diff) # (None, n_classes)
        logits = self.scale_logits(logits) # (None, n_classes)
        return logits
        
    def get_cosine(self, embeddings):
        """
        Args:
            embeddings: (None, embedding_size)
        Returns:
            cosine: (None, n_classes)
        """
        cosine = F.linear(F.normalize(embeddings), F.normalize(self.W))
        return cosine
    
    def get_target_mask(self, labels):
        """
        Args:
            labels: (None,)
        Returns:
            mask: (None, n_classes)
        """
        batch_size = labels.size(0)
        onehot = torch.zeros(batch_size, self.num_classes, device=labels.device)
        onehot.scatter_(1, labels.unsqueeze(-1), 1)
        return onehot
        
    def modify_cosine_of_target_classes(self, cosine_of_target_classes):
        """
        Args:
            cosine_of_target_classes: (None,)
        Returns:
            modified_cosine_of_target_classes: (None,)
        """
        eps = 1e-6
        # theta in the paper
        angles = torch.acos(torch.clamp(cosine_of_target_classes, -1 + eps, 1 - eps))
        return torch.cos(angles + self.margin)
    
    def scale_logits(self, logits):
        """
        Args:
            logits: (None, n_classes)
        Returns:
            scaled_logits: (None, n_classes)
        """
        return logits * self.scale
    
class SoftmaxLoss(nn.Module):
    def __init__(self, num_classes, embedding_size):
        """
        Regular softmax loss (1 fc layer without bias + CrossEntropyLoss)
        Args:
            num_classes: The number of classes in your training dataset
            embedding_size: The size of the embeddings that you pass into
        """
        super().__init__()
        self.num_classes = num_classes
        self.embedding_size = embedding_size
        
        self.W = torch.nn.Parameter(torch.Tensor(num_classes, embedding_size))
        nn.init.xavier_normal_(self.W)
        
    def forward(self, embeddings, labels):
        """
        Args:
            embeddings: (None, embedding_size)
            labels: (None,)
        Returns:
            loss: scalar
        """
        logits = F.linear(embeddings, self.W)
        return nn.CrossEntropyLoss()(logits, labels)

In [14]:
class Embedder(nn.Module):
    def __init__(self, embedding_size):
        super().__init__()
        self.model = torchvision.models.resnet50(pretrained = False)
        self.dropout = nn.Dropout(0.5)
        
        self.classifier = nn.Linear(1000, embedding_size)

    def forward(self, images):
        outputs = self.model(images)
        outputs = self.dropout(outputs)
        outputs = self.classifier(outputs)
        return outputs

In [15]:
embedding_size = 512
max_epochs = 100


embedder = Embedder(embedding_size=embedding_size)
arcface = ArcFaceLoss(num_classes=num_classe, embedding_size=embedding_size,margin=0.3, scale=30.0)

if torch.cuda.device_count() > 1:
    embedder = nn.DataParallel(embedder)
    arcface = nn.DataParallel(arcface)
embedder = embedder.to(device)
arcface = arcface.to(device)

optimizer = optim.Adam(embedder.parameters(), lr=1e-3 ) 
criterion = nn.CrossEntropyLoss()
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.9)


start = time.time()
for epoch in range(max_epochs):
    e_time = time.time()
    for i, (images, labels) in enumerate(trainloader):
        embedder.train()
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        embeddings = embedder(images)
        
        logits = arcface(embeddings, labels)
        loss = criterion(logits, labels)
        loss.backward() 
        optimizer.step() 
        
        if (i+1) % 64 == 0:
            print(f'Epoch: {epoch} - Batch: {i*batch_size} - Loss: {loss:.6f} - Time:{time.time() - e_time}')
            with open('/home/ielab/project/samples/mosaic_loss.txt', 'a') as file:
                file.write(f'{loss:.6f}\n')
            

/home/ielab/anaconda3/envs/arcface/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/ielab/anaconda3/envs/arcface/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch: 0 - Batch: 4032 - Loss: 17.163076 - Time:12.20831823348999
Epoch: 0 - Batch: 8128 - Loss: 17.195965 - Time:21.61734890937805
Epoch: 0 - Batch: 12224 - Loss: 17.280937 - Time:31.032713174819946
Epoch: 0 - Batch: 16320 - Loss: 17.035328 - Time:40.45094132423401
Epoch: 0 - Batch: 20416 - Loss: 17.044983 - Time:49.87278079986572
Epoch: 0 - Batch: 24512 - Loss: 16.852249 - Time:59.326844453811646
Epoch: 0 - Batch: 28608 - Loss: 17.301140 - Time:68.75272631645203
Epoch: 0 - Batch: 32704 - Loss: 17.026781 - Time:78.17005562782288
Epoch: 0 - Batch: 36800 - Loss: 17.130539 - Time:87.58853101730347
Epoch: 0 - Batch: 40896 - Loss: 17.096899 - Time:97.02509665489197
Epoch: 0 - Batch: 44992 - Loss: 17.061865 - Time:106.44508123397827
Epoch: 0 - Batch: 49088 - Loss: 17.392115 - Time:115.86147689819336
Epoch: 0 - Batch: 53184 - Loss: 17.149094 - Time:125.28547835350037
Epoch: 0 - Batch: 57280 - Loss: 17.105104 - Time:134.69430208206177
Epoch: 0 - Batch: 61376 - Loss: 17.180023 - Time:144.10358

Epoch: 3 - Batch: 49088 - Loss: 16.228437 - Time:113.12587404251099
Epoch: 3 - Batch: 53184 - Loss: 16.443521 - Time:122.53741312026978
Epoch: 3 - Batch: 57280 - Loss: 16.375132 - Time:131.95021271705627
Epoch: 3 - Batch: 61376 - Loss: 16.511751 - Time:141.36043286323547
Epoch: 3 - Batch: 65472 - Loss: 16.559866 - Time:150.7885901927948
Epoch: 3 - Batch: 69568 - Loss: 16.570635 - Time:160.19714426994324
Epoch: 3 - Batch: 73664 - Loss: 16.254227 - Time:169.60779118537903
Epoch: 3 - Batch: 77760 - Loss: 16.324650 - Time:179.0159149169922
Epoch: 3 - Batch: 81856 - Loss: 16.513859 - Time:188.43963408470154
Epoch: 3 - Batch: 85952 - Loss: 16.278278 - Time:197.84835958480835
Epoch: 3 - Batch: 90048 - Loss: 16.223141 - Time:207.25884413719177
Epoch: 3 - Batch: 94144 - Loss: 16.265440 - Time:216.68336844444275
Epoch: 3 - Batch: 98240 - Loss: 16.212202 - Time:226.09274911880493
Epoch: 3 - Batch: 102336 - Loss: 16.285799 - Time:235.50418853759766
Epoch: 3 - Batch: 106432 - Loss: 16.292656 - Time

Epoch: 6 - Batch: 90048 - Loss: 14.249108 - Time:207.23904633522034
Epoch: 6 - Batch: 94144 - Loss: 14.506726 - Time:216.65092945098877
Epoch: 6 - Batch: 98240 - Loss: 14.450743 - Time:226.0792200565338
Epoch: 6 - Batch: 102336 - Loss: 14.549614 - Time:235.49167466163635
Epoch: 6 - Batch: 106432 - Loss: 14.457609 - Time:244.90367937088013
Epoch: 6 - Batch: 110528 - Loss: 14.407684 - Time:254.31457328796387
Epoch: 6 - Batch: 114624 - Loss: 14.694676 - Time:263.72549867630005
Epoch: 6 - Batch: 118720 - Loss: 14.511046 - Time:273.15525579452515
Epoch: 6 - Batch: 122816 - Loss: 14.277832 - Time:282.56654143333435
Epoch: 6 - Batch: 126912 - Loss: 14.130533 - Time:291.99487042427063
Epoch: 6 - Batch: 131008 - Loss: 15.130928 - Time:301.40721344947815
Epoch: 6 - Batch: 135104 - Loss: 14.654873 - Time:310.81749272346497
Epoch: 6 - Batch: 139200 - Loss: 14.351925 - Time:320.22706389427185
Epoch: 6 - Batch: 143296 - Loss: 14.505547 - Time:329.63616728782654
Epoch: 6 - Batch: 147392 - Loss: 13.91

Epoch: 9 - Batch: 131008 - Loss: 9.877003 - Time:301.37659645080566
Epoch: 9 - Batch: 135104 - Loss: 9.013651 - Time:310.8027002811432
Epoch: 9 - Batch: 139200 - Loss: 9.868595 - Time:320.21410369873047
Epoch: 9 - Batch: 143296 - Loss: 8.680666 - Time:329.6259038448334
Epoch: 9 - Batch: 147392 - Loss: 8.013715 - Time:339.0517601966858
Epoch: 9 - Batch: 151488 - Loss: 9.716344 - Time:348.4626638889313
Epoch: 10 - Batch: 4032 - Loss: 7.584451 - Time:9.55242109298706
Epoch: 10 - Batch: 8128 - Loss: 8.783316 - Time:18.96183133125305
Epoch: 10 - Batch: 12224 - Loss: 8.085077 - Time:28.397619009017944
Epoch: 10 - Batch: 16320 - Loss: 7.589932 - Time:37.80693554878235
Epoch: 10 - Batch: 20416 - Loss: 8.757816 - Time:47.215375900268555
Epoch: 10 - Batch: 24512 - Loss: 8.867952 - Time:56.62265992164612
Epoch: 10 - Batch: 28608 - Loss: 7.653225 - Time:66.03135299682617
Epoch: 10 - Batch: 32704 - Loss: 7.712182 - Time:75.4392340183258
Epoch: 10 - Batch: 36800 - Loss: 7.784197 - Time:84.8492136001

...

Epoch: 85 - Batch: 32704 - Loss: 0.039104 - Time:75.49637842178345
Epoch: 85 - Batch: 36800 - Loss: 0.022437 - Time:84.91107702255249
Epoch: 85 - Batch: 40896 - Loss: 0.194298 - Time:94.3228690624237
Epoch: 85 - Batch: 44992 - Loss: 0.208708 - Time:103.73381805419922
Epoch: 85 - Batch: 49088 - Loss: 0.032618 - Time:113.14385986328125
Epoch: 85 - Batch: 53184 - Loss: 0.079302 - Time:122.55760097503662
Epoch: 85 - Batch: 57280 - Loss: 0.033627 - Time:131.9842336177826
Epoch: 85 - Batch: 61376 - Loss: 0.033689 - Time:141.39586353302002
Epoch: 85 - Batch: 65472 - Loss: 0.030375 - Time:150.80797839164734
Epoch: 85 - Batch: 69568 - Loss: 0.028511 - Time:160.21960639953613
Epoch: 85 - Batch: 73664 - Loss: 0.029791 - Time:169.63049960136414
Epoch: 85 - Batch: 77760 - Loss: 0.047236 - Time:179.06203532218933
Epoch: 85 - Batch: 81856 - Loss: 0.054007 - Time:188.47618460655212
Epoch: 85 - Batch: 85952 - Loss: 0.074729 - Time:197.88746309280396
Epoch: 85 - Batch: 90048 - Loss: 0.856875 - Time:207.

Epoch: 88 - Batch: 73664 - Loss: 0.199038 - Time:169.6334810256958
Epoch: 88 - Batch: 77760 - Loss: 0.218985 - Time:179.04432344436646
Epoch: 88 - Batch: 81856 - Loss: 0.108210 - Time:188.4551386833191
Epoch: 88 - Batch: 85952 - Loss: 0.028952 - Time:197.86748719215393
Epoch: 88 - Batch: 90048 - Loss: 0.342223 - Time:207.2802538871765
Epoch: 88 - Batch: 94144 - Loss: 0.019517 - Time:216.7075366973877
Epoch: 88 - Batch: 98240 - Loss: 0.029686 - Time:226.1195547580719
Epoch: 88 - Batch: 102336 - Loss: 0.073261 - Time:235.53033065795898
Epoch: 88 - Batch: 106432 - Loss: 0.217617 - Time:244.94371366500854
Epoch: 88 - Batch: 110528 - Loss: 0.124855 - Time:254.35665369033813
Epoch: 88 - Batch: 114624 - Loss: 0.064393 - Time:263.78671646118164
Epoch: 88 - Batch: 118720 - Loss: 0.115821 - Time:273.19865822792053
Epoch: 88 - Batch: 122816 - Loss: 0.155418 - Time:282.62674927711487
Epoch: 88 - Batch: 126912 - Loss: 0.509344 - Time:292.03842067718506
Epoch: 88 - Batch: 131008 - Loss: 0.046400 - T

Epoch: 91 - Batch: 114624 - Loss: 0.024579 - Time:263.7205505371094
Epoch: 91 - Batch: 118720 - Loss: 0.075275 - Time:273.14756417274475
Epoch: 91 - Batch: 122816 - Loss: 0.020097 - Time:282.5588870048523
Epoch: 91 - Batch: 126912 - Loss: 0.227313 - Time:291.98433017730713
Epoch: 91 - Batch: 131008 - Loss: 0.019901 - Time:301.3958423137665
Epoch: 91 - Batch: 135104 - Loss: 0.054942 - Time:310.8061935901642
Epoch: 91 - Batch: 139200 - Loss: 0.041509 - Time:320.2151930332184
Epoch: 91 - Batch: 143296 - Loss: 0.047840 - Time:329.6225917339325
Epoch: 91 - Batch: 147392 - Loss: 0.123527 - Time:339.03097438812256
Epoch: 91 - Batch: 151488 - Loss: 0.039056 - Time:348.4404032230377
Epoch: 92 - Batch: 4032 - Loss: 0.026664 - Time:9.536698579788208
Epoch: 92 - Batch: 8128 - Loss: 0.025687 - Time:18.975086450576782
Epoch: 92 - Batch: 12224 - Loss: 0.035156 - Time:28.387681245803833
Epoch: 92 - Batch: 16320 - Loss: 0.031470 - Time:37.7989444732666
Epoch: 92 - Batch: 20416 - Loss: 0.022725 - Time:4

Epoch: 95 - Batch: 4032 - Loss: 0.041163 - Time:9.55743670463562
Epoch: 95 - Batch: 8128 - Loss: 0.102504 - Time:18.99421501159668
Epoch: 95 - Batch: 12224 - Loss: 0.064593 - Time:28.40384078025818
Epoch: 95 - Batch: 16320 - Loss: 0.027282 - Time:37.81459856033325
Epoch: 95 - Batch: 20416 - Loss: 0.027565 - Time:47.22298455238342
Epoch: 95 - Batch: 24512 - Loss: 0.019421 - Time:56.63504242897034
Epoch: 95 - Batch: 28608 - Loss: 0.031265 - Time:66.04613161087036
Epoch: 95 - Batch: 32704 - Loss: 0.111991 - Time:75.45693707466125
Epoch: 95 - Batch: 36800 - Loss: 0.112890 - Time:84.86908197402954
Epoch: 95 - Batch: 40896 - Loss: 0.280081 - Time:94.29621171951294
Epoch: 95 - Batch: 44992 - Loss: 0.029152 - Time:103.7048659324646
Epoch: 95 - Batch: 49088 - Loss: 0.034049 - Time:113.11289429664612
Epoch: 95 - Batch: 53184 - Loss: 0.027209 - Time:122.52015399932861
Epoch: 95 - Batch: 57280 - Loss: 0.125081 - Time:131.9458725452423
Epoch: 95 - Batch: 61376 - Loss: 0.076200 - Time:141.3541784286

Epoch: 98 - Batch: 49088 - Loss: 0.100937 - Time:113.11602783203125
Epoch: 98 - Batch: 53184 - Loss: 0.069299 - Time:122.52554821968079
Epoch: 98 - Batch: 57280 - Loss: 0.029761 - Time:131.93733048439026
Epoch: 98 - Batch: 61376 - Loss: 0.025317 - Time:141.36485862731934
Epoch: 98 - Batch: 65472 - Loss: 0.085758 - Time:150.7747769355774
Epoch: 98 - Batch: 69568 - Loss: 0.075758 - Time:160.1845464706421
Epoch: 98 - Batch: 73664 - Loss: 0.062395 - Time:169.60935044288635
Epoch: 98 - Batch: 77760 - Loss: 0.025425 - Time:179.0172302722931
Epoch: 98 - Batch: 81856 - Loss: 0.043488 - Time:188.42880058288574
Epoch: 98 - Batch: 85952 - Loss: 0.037840 - Time:197.84016180038452
Epoch: 98 - Batch: 90048 - Loss: 0.029996 - Time:207.2477638721466
Epoch: 98 - Batch: 94144 - Loss: 0.187321 - Time:216.65730166435242
Epoch: 98 - Batch: 98240 - Loss: 0.038482 - Time:226.06885528564453
Epoch: 98 - Batch: 102336 - Loss: 0.122628 - Time:235.47984552383423
Epoch: 98 - Batch: 106432 - Loss: 0.014225 - Time:2

In [17]:
#  정확도 측정
train_results = []
train_labels = []
test_results = []
test_labels = []

embedder.eval()
with torch.no_grad():
  for img, label in trainloader:
    img = img.cuda()
    train_results.append(embedder(img).cpu().detach().numpy())
    train_labels.append(label)

train_results = np.concatenate(train_results)
train_labels = np.concatenate(train_labels)

embedder.eval()
with torch.no_grad():
  for img, label in testloader:
    img = img.cuda()
    test_results.append(embedder(img).cpu().detach().numpy())
    test_labels.append(label)

test_results = np.concatenate(test_results)
test_labels = np.concatenate(test_labels)

In [18]:
from sklearn.neighbors import KNeighborsClassifier


# kNN 모델 선언
k = 50
model = KNeighborsClassifier(n_neighbors = k)
# 모델 학습
model.fit(train_results, train_labels)
#knn 검증
train_pred = model.predict(train_results)
train_acc = (train_pred == train_labels).mean()
with open('/home/ielab/project/samples/gan_train_acc.txt', 'a') as file:
    file.write(f'{train_acc:.6f}\n')
print(f'train acc : {train_acc}')
test_pred = model.predict(test_results)
test_acc = (test_pred == test_labels).mean()
with open('/home/ielab/project/samples/gan_test_acc.txt', 'a') as file:
    file.write(f'{test_acc:.6f}\n')
print(f'test acc : {test_acc}')


train acc : 0.9777071715730774
test acc : 0.3052923923482233


NameError: name 'metrics' is not defined

In [20]:
# F1-Score
f1 = metrics.classification_report(test_labels, test_pred)
with open('/home/ielab/project/samples/gan_test_f1.txt', 'a') as file:
    file.write(f'{f1}\n')

/home/ielab/anaconda3/envs/arcface/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ielab/anaconda3/envs/arcface/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ielab/anaconda3/envs/arcface/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

In [21]:
print(f1)

              precision    recall  f1-score   support

           0       1.00      0.23      0.38        13
           1       0.00      0.00      0.00        19
           2       0.67      0.38      0.48        21
           3       0.00      0.00      0.00         1
           4       1.00      0.33      0.50         6
           5       0.00      0.00      0.00        15
           6       0.00      0.00      0.00         2
           7       1.00      0.42      0.59        19
           8       1.00      0.17      0.29        24
           9       0.13      0.50      0.21        52
          10       1.00      0.05      0.10        20
          11       0.80      0.24      0.36        17
          12       0.89      0.89      0.89         9
          13       1.00      0.29      0.44        14
          14       0.00      0.00      0.00        10
          15       1.00      0.41      0.58        17
          16       0.00      0.00      0.00         5
          17       0.00    

In [ ]:
og_test_path = '/home/ielab/dataset/ms1m_dataset/test'

og_testset = CustomDataset(og_test_path, trans)
og_testloader = DataLoader(og_testset, batch_size=batch_size, shuffle=False, num_workers=4)

In [ ]:
og_test_results = []
og_test_labels = []

embedder.eval()
with torch.no_grad():
  for img, label in og_testloader:
    img = img.cuda()
    og_test_results.append(embedder(img).cpu().detach().numpy())
    og_test_labels.append(label)
        
og_test_results = np.concatenate(og_test_results)
og_test_labels = np.concatenate(og_test_labels)

In [ ]:
og_test_pred = model.predict(og_test_results)
(og_test_pred == og_test_labels).mean()

In [ ]:
torch.save(embedder, './mosaic_model.pth')